<h1>Data exploration and preparation</h1>

In this and the following notebooks we will demonstrate how you can build your ML Pipeline leveraging Spark Feature Transformers and SageMaker XGBoost algorithm & after the model is trained, deploy the Pipeline (Feature Transformer and XGBoost) as an Inference Pipeline behind a single Endpoint for real-time inference and for batch inferences using Amazon SageMaker Batch Transform.

In particular, in this notebook we will tackle the first steps related to data exploration and preparation. We will use [Amazon Athena](https://aws.amazon.com/athena/) to query our dataset and have a first insight about data quality and available features, and [AWS Glue](https://aws.amazon.com/glue/) to create a Data Catalog and run serverless Spark jobs.

<span style="color: red">**To get started, in the cell below please replace your initials in the bucket_name variable, in order to match the bucket name you've created in the previous steps.**</span>

In [ ]:
import boto3
import sagemaker
import time

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

print(region)
print(role)
 
# replace [your-initials] according to the bucket name you have defined.
bucket_name = 'endtoendml-workshop-[your-initials]'

print(bucket_name)

We can now copy to our bucket the dataset used for this use case. We will use the `windturbine_raw_data.csv` made available for this workshop in the `gianpo-public` public S3 bucket. In this Notebook, we will download from that bucket and upload to your bucket so that AWS Glue can access the data.

In [ ]:
import boto3

s3 = boto3.resource('s3')

copy_source = {
    'Bucket': 'gianpo-public',
    'Key': 'windturbine_raw_data.csv'
}

file_name = 'windturbine_raw_data.csv'
file_key = 'data/{0}'.format(file_name)
s3.Bucket(bucket_name).object_versions.delete()
s3.Bucket(bucket_name).copy(copy_source, file_key)

The first thing we need now is to infer a schema for our dataset. Thanks to its [integration with AWS Glue](https://docs.aws.amazon.com/athena/latest/ug/glue-athena.html), we will later use Amazon Athena to run SQL queries against our data stored in S3 without the need to import them into a relational database. To do so, Amazon Athena uses the AWS Glue Data Catalog as a central location to store and retrieve table metadata throughout an AWS account. The Athena execution engine, indeed, requires table metadata that instructs it where to read data, how to read it, and other information necessary to process the data.

To organize our Glue Data Catalog we create a new database named `endtoendml-db`. To do so, we create a Glue client via Boto and invoke the `create_database` method.

However, first we want to make sure these objects to not exist yet to avoid any error.

In [ ]:
glue_client = boto3.client('glue')

# Trying to remove any existing database/crawler with the same name.

crawler_found = True
try:
    glue_client.get_crawler(Name = 'endtoendml-crawler')
except glue_client.exceptions.EntityNotFoundException:
    crawler_found = False

db_found = True
try:
    glue_client.get_database(Name = 'endtoendml-db')
except glue_client.exceptions.EntityNotFoundException:
    db_found = False

if crawler_found:
    glue_client.delete_crawler(Name = 'endtoendml-crawler')
if db_found:
     glue_client.delete_database(Name = 'endtoendml-db')

print("Cleanup completed.")

In [ ]:
response = glue_client.create_database(DatabaseInput={'Name': 'endtoendml-db'})
response = glue_client.get_database(Name='endtoendml-db')
response
assert response['Database']['Name'] == 'endtoendml-db'

Now we define a Glue Crawler that we point to the S3 path where the dataset resides, and the crawler creates table definitions in the Data Catalog.
To grant the correct set of access permission to the crawler, we use one of the roles created before (`GlueServiceRole-endtoendml`) whose policy grants AWS Glue access to data stored in your S3 buckets.

In [ ]:
response = glue_client.create_crawler(
    Name='endtoendml-crawler',
    Role='service-role/GlueServiceRole-endtoendml', 
    DatabaseName='endtoendml-db',
    Targets={'S3Targets': [{'Path': '{0}/data/'.format(bucket_name)}]}
)

We are ready to run the crawler with the `start_crawler` API and to monitor its status upon completion through the `get_crawler_metrics` API.

In [ ]:
glue_client.start_crawler(Name='endtoendml-crawler')

while glue_client.get_crawler_metrics(CrawlerNameList=['endtoendml-crawler'])['CrawlerMetricsList'][0]['TablesCreated'] == 0:
    print('RUNNING')
    time.sleep(15)
    
assert glue_client.get_crawler_metrics(CrawlerNameList=['endtoendml-crawler'])['CrawlerMetricsList'][0]['TablesCreated'] == 1


When the crawler has finished its job, we can retrieve the Table definition for the newly created table.
As you can see, the crawler has been able to correctly identify 12 fields, infer a type for each column and assign a name.

In [ ]:
table = glue_client.get_table(DatabaseName='endtoendml-db', Name='data')
table

Based on our knowledge of the dataset, we can assign more specific names to columns.

In [ ]:
table['Table']['StorageDescriptor']['Columns'] = [{'Name': 'turbine_id', 'Type': 'string'},
                                                  {'Name': 'turbine_type', 'Type': 'string'},
                                                  {'Name': 'wind_speed', 'Type': 'double'},
                                                  {'Name': 'rpm_blade', 'Type': 'double'},
                                                  {'Name': 'oil_temperature', 'Type': 'double'},
                                                  {'Name': 'oil_level', 'Type': 'double'},
                                                  {'Name': 'temperature', 'Type': 'double'},
                                                  {'Name': 'humidity', 'Type': 'double'},
                                                  {'Name': 'vibrations_frequency', 'Type': 'double'},
                                                  {'Name': 'pressure', 'Type': 'double'},
                                                  {'Name': 'wind_direction', 'Type': 'string'},
                                                  {'Name': 'breakdown', 'Type': 'string'}]
updated_table = table['Table']
updated_table.pop('DatabaseName', None)
updated_table.pop('CreateTime', None)
updated_table.pop('UpdateTime', None)
updated_table.pop('CreatedBy', None)
updated_table.pop('IsRegisteredWithLakeFormation', None)

glue_client.update_table(
    DatabaseName='endtoendml-db',
    TableInput=updated_table
)

In [ ]:
!pip install pyathena

In [ ]:
import pyathena
from pyathena import connect
from pyathena.pandas_cursor import PandasCursor
import pandas as pd

conn = connect(s3_staging_dir='s3://{0}/staging/'.format(bucket_name), 
               region_name=region, cursor_class=PandasCursor)

df = pd.read_sql('SELECT * FROM "endtoendml-db".data limit 8;', conn)
df

Another SQL query to count how many records we have

In [ ]:
pd.read_sql('SELECT COUNT(*) FROM "endtoendml-db".data;', conn)

Let's try to see what are possible values for the field "alarm" and how frequently they occur over the entire dataset

In [ ]:
pd.read_sql('SELECT breakdown, (COUNT(breakdown) * 100.0 / (SELECT COUNT(*) FROM "endtoendml-db".data)) \
            AS percent FROM "endtoendml-db".data GROUP BY breakdown;', conn)


In [ ]:
pd.read_sql('SELECT DISTINCT(turbine_type) FROM "endtoendml-db".data', conn)

In [ ]:
pd.read_sql('SELECT COUNT(*) FROM "endtoendml-db".data WHERE oil_temperature IS NULL GROUP BY oil_temperature', conn)

Now we want to see if there is a correlation between temperature and humidity. To do so we run a SQL query to select only these two columns and populate a Pandas dataframe that we will use for our analysis

In [ ]:
temp_hum_df = pd.read_sql('SELECT temperature, humidity FROM "endtoendml-db".data', conn)
temp_hum_df.head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.scatter(temp_hum_df.temperature, temp_hum_df.humidity)

In [ ]:
plt.hist(temp_hum_df.humidity, bins=10)

In [ ]:
wind_rpm_df = pd.read_sql('SELECT wind_speed, rpm_blade FROM "endtoendml-db".data', conn)
plt.scatter(wind_rpm_df.wind_speed, wind_rpm_df.rpm_blade)

Note: you can go to Amazon Athena console and check for query duration under History tab: usually queries are executed in a few seconds, then it takes a while for pandas to load results into a dataframe

In [ ]:
wind_rpm_df.describe()

Now we select our entire dataset and populate a dataframe.  

In [ ]:
df = pd.read_sql('SELECT * FROM "endtoendml-db".data;', conn)
df.info()

You can notice that col4float has some missing values

In [ ]:
df.describe(include=['object', 'int64', 'float64'])

In [ ]:
!wget https://s3-us-west-2.amazonaws.com/sparkml-mleap/0.9.6/python/python.zip
!wget https://s3-us-west-2.amazonaws.com/sparkml-mleap/0.9.6/jar/mleap_spark_assembly.jar

In [ ]:
s3.Bucket(bucket_name).upload_file('python.zip', 'dependencies/python/python.zip')
s3.Bucket(bucket_name).upload_file('mleap_spark_assembly.jar', 'dependencies/jar/mleap_spark_assembly.jar')

In [ ]:
!pygmentize endtoendml_etl.py

In [ ]:
s3.Bucket(bucket_name).upload_file('endtoendml_etl.py', 'code/endtoendml_etl.py')

ETLJob = glue_client.create_job(Name='endtoendml-job', 
                                Role='GlueServiceRole-endtoendml',
                                Command={
                                    'Name': 'glueetl',
                                    'ScriptLocation': 's3://{0}/code/endtoendml_etl.py'.format(bucket_name)
                                },
                               DefaultArguments={
                                   '--job-language': 'python',
                                   '--extra-jars' : 's3://{0}/dependencies/jar/mleap_spark_assembly.jar'.format(bucket_name),
                                   '--extra-py-files': 's3://{0}/dependencies/python/python.zip'.format(bucket_name)
                               })
glue_job_name = ETLJob['Name']
print(glue_job_name)

In [ ]:
JobRun = glue_client.start_job_run(JobName=glue_job_name, 
                                  Arguments = {'--S3_BUCKET': bucket_name})
print(JobRun)


In [ ]:
status = glue_client.get_job_run(JobName=ETLJob['Name'], RunId=JobRun['JobRunId'])
while status['JobRun']['JobRunState'] not in ('FAILED', 'SUCCEEDED', 'STOPPED'):
    print('Job status: ' + status['JobRun']['JobRunState'])
    time.sleep(30)
    status = glue_client.get_job_run(JobName=ETLJob['Name'], RunId=JobRun['JobRunId'])

print(status['JobRun']['JobRunState'])
    
#This will take around 15 minutes

After the job is completed, you can move to the next notebook in the **03_train_model** folder to start model training.